<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [ ]:
!pip install fold_to_ascii
!pip install stanza
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
from collections import Counter
from fold_to_ascii import fold
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
import nltk
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import tweepy
from nltk import pos_tag
from nltk import RegexpParser
import spacy
from spacy import displacy
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7dde3047ad782922eeded826717c4e8abf45f12410497b04f555e09e2e946827
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Write your code here

consumer_key ="2Em7SxlX9jPMfL4x97r3zMO0x"
consumer_secret = "sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd"
access_key ="1439767876962029572-uUMt8oWRyzj9ilE5zk4uYbL93sCMPT"
access_secret = "oydIGymn9bS767FVEMawE9GyGAnMmBJfaY2XXKmHnmliF"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
df = pd.DataFrame(columns=['username', 'text'])
print("Enter Twitter HashTag to search for")
words = input()
tweet_data = tweepy.Cursor(api.search,words, lang="en",tweet_mode='extended').items(1000)
tweet_list = list()
for i in tweet_data:
  tweet_list.append(i)
for tweet in tweet_list:
  try:
    username = tweet.user.screen_name
    text = tweet.retweeted_status.full_text
  except AttributeError:
    text = tweet.full_text
    username = tweet.user.screen_name
  df.loc[len(df)] = [username,text]
df


Enter Twitter HashTag to search for
#music


,username,text
0,righettirigo,with my @franchinguitars Rigo's signature bass...
1,sauparna_manna,My New Work as Music Director\nMess A Massacre...
2,the_eighth_room,"#NowPlaying Dandy Livingstone - Rudy, A Messag..."
3,dhanushfani,The Director Who Took Tamil Cinema to New Heig...
4,wnmcradio,"Now Playing, Rahzel, Shaggy, Toots &amp; the M..."
...,...,...
995,2018Dear,@Fanclub_phoenix Dimash shares his beautiful f...
996,Slivslk,Song making up! #music #Succession #artist #lo...
997,chkl8dva,George Michael - Father Figure (1987) 🎶🎶🎶🎶🎶\n\...
998,RadioMetalRock,MILLENNIUM - Assassin via #PureSteel #Playing ...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [ ]:
# Write your code here

def finding_ex_c(row): #function for finding special characters
  l = list()
  a=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','\t',' ','\n']
  for i in df[row]:
    if i == 0:
      continue
    for j in i:
      j = fold(str(j))
      k=j.lower()
      for m in k:
        if(m not in a):
          l.append(m)
  return set(l)



In [ ]:
def text_clean(col, mp): #function for text cleaning
  stop_words = stopwords.words('english')
  stop_words += ['why', 'where', 'what', 'when', 'who']
  title = list()
  x = ['0','2','1','3','4','5','6','7','8','9']
  for lis in df[col]:
    if lis == 0:
      title.append(0)
      continue
    y = list()
    word_lis = lis.split(" ")
    for words in word_lis :
      words = fold(words)
      if words == '':
        continue
      elif words.isdigit():
        continue
      else:
        w = ''
        for c in words:
          if c in x:
            w+= ' '
          elif c in mp:
            w += ' '
          else:
            w += c
        w = w.strip(" ")
        w = w.lower()
        if  len(w) > 0 and w not in stop_words:
          if w not in y:
            y.append(w.strip(" "))
    title.append(y)
  return title

In [ ]:
sepcial_characters = finding_ex_c('text')
sepcial_characters

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 '{',
 '|',
 '}',
 '~'}

In [ ]:
clean_text = text_clean('text',sepcial_characters )

In [ ]:
df['text'] = clean_text
df

,username,text
0,righettirigo,"[franchinguitars, rigo s, signature, bass, cal..."
1,sauparna_manna,"[new, work, music, director\nmess, massacre, w..."
2,the_eighth_room,"[nowplaying, dandy, livingstone, rudy, message..."
3,dhanushfani,"[director, took, tamil, cinema, new, heights \..."
4,wnmcradio,"[playing, rahzel, shaggy, toots, amp, maytals,..."
...,...,...
995,2018Dear,"[fanclub phoenix, dimash, shares, beautiful, f..."
996,Slivslk,"[song, making, music, succession, artist, love..."
997,chkl8dva,"[george, michael, father, figure, \n\n georgem..."
998,RadioMetalRock,"[millennium, assassin, via, puresteel, playing..."


In [ ]:
#stemming
stemer = PorterStemmer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
   p.append(stemer.stem(word))
  l.append(p)
df['text'] = l
df

,username,text
0,righettirigo,"[franchinguitar, rigo , signatur, bass, call, ..."
1,sauparna_manna,"[new, work, music, director\nmess, massacr, we..."
2,the_eighth_room,"[nowplay, dandi, livingston, rudi, messag, eig..."
3,dhanushfani,"[director, took, tamil, cinema, new, heights \..."
4,wnmcradio,"[play, rahzel, shaggi, toot, amp, maytal, bam,..."
...,...,...
995,2018Dear,"[fanclub phoenix, dimash, share, beauti, feel,..."
996,Slivslk,"[song, make, music, success, artist, love, htt..."
997,chkl8dva,"[georg, michael, father, figur, \n\n georgemic..."
998,RadioMetalRock,"[millennium, assassin, via, puresteel, play, m..."


In [ ]:
# Lemmatization
lemma = WordNetLemmatizer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
    p.append(lemma.lemmatize(word, pos='v'))
  l.append(p)
df['text'] = l
df

,username,text
0,righettirigo,"[franchinguitar, rigo , signatur, bass, call, ..."
1,sauparna_manna,"[new, work, music, director\nmess, massacr, we..."
2,the_eighth_room,"[nowplay, dandi, livingston, rudi, messag, eig..."
3,dhanushfani,"[director, take, tamil, cinema, new, heights \..."
4,wnmcradio,"[play, rahzel, shaggi, toot, amp, maytal, bam,..."
...,...,...
995,2018Dear,"[fanclub phoenix, dimash, share, beauti, feel,..."
996,Slivslk,"[song, make, music, success, artist, love, htt..."
997,chkl8dva,"[georg, michael, father, figur, \n\n georgemic..."
998,RadioMetalRock,"[millennium, assassin, via, puresteel, play, m..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# POS
c = Counter()
for text in df['text']:
  tokens_tag = pos_tag(text)
  print(tokens_tag)
  for i in tokens_tag:
    c[i[1]] += 1




[('franchinguitar', 'NN'), ('rigo ', 'NN'), ('signatur', 'JJ'), ('bass', 'NN'), ('call', 'NN'), ('new', 'JJ'), ('gold', 'NN'), ('dream', 'NN'), ('live \n gold', 'VBD'), ('electr', 'JJ'), ('music', 'NN'), ('groov', 'NN'), ('musicisunivers', 'NNS'), ('musica', 'VBP'), ('rigorighetti', 'JJ'), ('vibesnight', 'NN'), ('rigorecord', 'NN'), ('ampeg', 'NN'), ('ampeg jp', 'NN'), ('righettirigo', 'NN'), ('https   t co zmrhyvlf', 'NN')]
[('new', 'JJ'), ('work', 'NN'), ('music', 'NN'), ('director\nmess', 'NN'), ('massacr', 'NN'), ('web', 'NN'), ('seri', 'NN'), ('\nwill', 'NNP'), ('avail', 'VBZ'), ('youtube\n webseri', 'NN'), ('bangla', 'NN'), ('bengaliseri', 'NN'), ('movi', 'NN'), ('youtub', 'NN'), ('messamassacr', 'NN'), ('https   t co  x tv xwi', 'NN')]
[('nowplay', 'NN'), ('dandi', 'VBZ'), ('livingston', 'JJ'), ('rudi', 'NN'), ('messag', 'NN'), ('eighth', 'JJ'), ('room', 'NN'), ('stream', 'NN'), ('live', 'JJ'), ('https   t co  dw t n l', 'NN'), ('nashvil', 'JJ'), ('music', 'NN'), ('loung', 'NN')

In [ ]:
#count for each POS element
print(c)

Counter({'NN': 11392, 'JJ': 2221, 'VBP': 707, 'NNP': 553, 'VB': 285, 'NNS': 212, 'IN': 204, 'RB': 203, 'VBZ': 158, 'VBD': 148, 'FW': 66, 'JJS': 51, 'CD': 38, 'VBN': 37, 'PRP': 25, 'MD': 16, 'JJR': 14, 'VBG': 10, 'RP': 8, 'RBR': 8, 'DT': 7, 'CC': 6, 'RBS': 4, 'WP': 3, 'WRB': 3, 'WP$': 3, 'WDT': 2, "''": 1, 'TO': 1})


In [ ]:
# dependency parsing
nlp=spacy.load('en_core_web_sm')

for text in df['text']: 
  sen = ""
  for j in text:
    sen = sen+j+" "
  dc = nlp(sen)
  for i in dc:
    print("Token\t\t|Relation\t\t|Head\t\t|Children")
    print("-------------------------------------------")
    print(i.text,"\t\t|",i.dep_,"\t\t|",i.head.text,"\t\t|",[j for i in i.children])
  print("\n")

Streaming output truncated to the last 5000 lines.
iowok 		| compound 		| u 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
u 		| ROOT 		| u 		| ['https   t co iowok u ji', 'https   t co iowok u ji']
Token		|Relation		|Head		|Children
-------------------------------------------
ji 		| punct 		| u 		| []


Token		|Relation		|Head		|Children
-------------------------------------------
sv 		| compound 		| product 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
product 		| nsubj 		| citi 		| ['https   t co dktdd zxei']
Token		|Relation		|Head		|Children
-------------------------------------------
even 		| advmod 		| citi 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
citi 		| ROOT 		| citi 		| ['https   t co dktdd zxei', 'https   t co dktdd zxei', 'https   t co dktdd zxei', 'https   t co dktdd zxei']
Token		|Relation		|Head		|Children
------------------------------------------

In [ ]:
# Consistency parsing
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
for text in df['text']:
  rp = ''
  for j in df['text'][0]:
    rp = rp+' '+j
  doc = nlp(rp)
  for sentence in doc.sentences:
      print(sentence.constituency)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


(ROOT (S (NP (FW franchinguitar) (FW rigo) (FW signatur) (FW bass)) (VP (FW call) (S (NP (JJ new) (NN gold) (NN dream)) (VP (FW live) (NP (NML (NN gold) (FW electr) (FW music)) (FW groov) (FW musicisunivers) (FW musica) (FW rigorighetti) (FW vibesnight) (FW rigorecord) (FW ampeg) (FW ampeg) (FW jp) (GW righettirigo) (GW https) (GW t) (FW co) (FW zmrhyvlf)))))))
(ROOT (S (NP (FW franchinguitar) (FW rigo) (FW signatur) (FW bass)) (VP (FW call) (S (NP (JJ new) (NN gold) (NN dream)) (VP (FW live) (NP (NML (NN gold) (FW electr) (FW music)) (FW groov) (FW musicisunivers) (FW musica) (FW rigorighetti) (FW vibesnight) (FW rigorecord) (FW ampeg) (FW ampeg) (FW jp) (GW righettirigo) (GW https) (GW t) (FW co) (FW zmrhyvlf)))))))
(ROOT (S (NP (FW franchinguitar) (FW rigo) (FW signatur) (FW bass)) (VP (FW call) (S (NP (JJ new) (NN gold) (NN dream)) (VP (FW live) (NP (NML (NN gold) (FW electr) (FW music)) (FW groov) (FW musicisunivers) (FW musica) (FW rigorighetti) (FW vibesnight) (FW rigorecord) (F

In [ ]:
# named entity recognition
NER = spacy.load("en_core_web_sm")
for text in df['text']:
  rp = ''
  for j in text:
    rp = rp+' '+j
  doc = NER(rp)
  for sentence in doc.ents:
      print("Token\t\t|Entity")
      print("-----------------")
      print(sentence.text,"\t|",sentence.label_)

Streaming output truncated to the last 5000 lines.
-----------------
https 	| PERSON
Token		|Entity
-----------------
activ forc 	| PERSON
Token		|Entity
-----------------
lover https 	| PERSON
Token		|Entity
-----------------
coo cal 	| ORG
Token		|Entity
-----------------
michael 	| PERSON
Token		|Entity
-----------------
lisa stansfield 	| PERSON
Token		|Entity
-----------------
freddiemercuri 	| PERSON
Token		|Entity
-----------------
new week 	| DATE
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
motivationalquot 	| PERSON
Token		|Entity
-----------------
darren 	| PERSON
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
one 	| CARDINAL
Token		|Entity
-----------------
english 	| NORP
Token		|Entity
-----------------
hindi tajikistan 	| PERSON
Token		|Entity
-----------------
uae dubai instalik studio new song love life https 	| ORG
Token		|Entity
-----------------
frat

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency Parsing Tree:
Using a constituency parsing tree, a sentence's syntactic structure can be represented in terms of its individual phrases. The tree is constructed by recursively breaking down a sentence into smaller components, or constituents, which are then grouped together to form larger constituents. Until the entire sentence is covered, this process is repeated. The sentence as a whole is represented by the tree's trunk, and the individual syllables are represented by the leaves. Each internal node of the tree is a representation of a constituent phrase, which is made up of one or more smaller constituents.

Dependency Parsing Tree:
The syntactic structure of a phrase can also be represented using a dependency parsing tree, although this method shows the relationships between words in terms of their dependencies rather than classifying them according to their components. Each word in a sentence is represented as a node in a dependency tree, and the connections between words are shown as directed edges connecting the nodes. A grammatical relationship, such as a subject, object, or modifier, is represented by each edge.
